In [324]:
from nba_api.stats.endpoints import shotchartdetail, videodetails
import json
import requests
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

In [325]:
headers = {
    'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'x-nba-stats-origin': 'stats',
    'x-nba-stats-token': 'true',
    'Connection': 'keep-alive',
    'Origin': 'https://stats.nba.com',
    'Referer': 'https://stats.nba.com/',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache'
}

In [326]:
franz_wuerfe = pd.read_csv('franz.csv', dtype ='str')
gameids = franz_wuerfe["GAME_ID"].tolist()
gameevents = franz_wuerfe["GAME_EVENT_ID"].tolist()
franz_wuerfe[['GAME_ID','GAME_EVENT_ID']] = franz_wuerfe[['GAME_ID','GAME_EVENT_ID']].astype('int64')


In [327]:
franz_wuerfe_neu= len(franz_wuerfe.index)
franz_wuerfe_alt = pd.read_csv('franz_wuerfe_videos.csv')
franz_wuerfe_alt= len(franz_wuerfe_alt.index)
neue_wuerfe =  franz_wuerfe_neu-franz_wuerfe_alt


In [328]:
index = 0
if neue_wuerfe > 0:
    franz_wuerfe_video =[]
    for gameid, gameevent in zip(gameids[-int(neue_wuerfe):], gameevents[-int(neue_wuerfe):]):
        time.sleep(.6)
        index += 1
        print(index)
        event_id = gameevent
        game_id =  gameid
        url = 'https://stats.nba.com/stats/videoeventsasset?GameEventID={}&GameID={}'.format(
                    event_id, game_id)
        r = requests.get(url, headers=headers)
        json = r.json()
        video_urls = json['resultSets']['Meta']['videoUrls']
        GameID = json['parameters']['GameID']
        GameEventID = json['parameters']['GameEventID']
        video_event = {'video': video_urls[0]['lurl'], 'game_id': GameID, 'event_id':GameEventID}
        video_event=pd.DataFrame.from_dict(video_event,orient='index').T
        franz_wuerfe_video.append(video_event)
    franz_wuerfe_video = pd.concat([video_event for video_event in franz_wuerfe_video])
    franz_wuerfe_video.to_csv('franz_videos.csv',index=False)

    franz_wuerfe_video = franz_wuerfe_video.rename(columns={'game_id': 'GAME_ID', 'event_id': 'GAME_EVENT_ID'})
    franz_wuerfe_video[['GAME_ID','GAME_EVENT_ID']] = franz_wuerfe_video[['GAME_ID','GAME_EVENT_ID']].astype('int64')
    franz_wuerfe_video_merge = pd.merge(franz_wuerfe_video, franz_wuerfe, on=['GAME_ID','GAME_EVENT_ID'],  how='outer')
    franz_wuerfe_video_merge.to_csv('franz_wuerfe_videos.csv',mode="a", header="false", index=False)
else:
    print("Keine neuen Würfe")

Keine neuen Würfe
